## Importing all the required libraries 

In [ ]:
import pandas 
import numpy
import glob
import gensim
import logging
import sys, os
import nltk
import itertools
import multiprocessing 

from time import time  # To time our operations
from gensim import models

# */site-packages is where your current session is running its python out of
site_path = ''
for path in sys.path:
    if 'site-packages' in path.split('/')[-1]:
        print(path)
        site_path = path
# search to see if gensim in installed packages
if len(site_path) > 0:
    if not 'gensim' in os.listdir(site_path):
        print('package not found')
    else:
        print('gensim installed')    

## Reading all the text files in the corpus and tokeniztion

In [ ]:
def readCorpus():
    #logging.info("reading file {0}...this may take a while".format(input_file))
    for filename in glob.glob('Train/*.txt'):
        print(filename)
        with open(filename, 'rb') as f:
            line = f.read()
            return nltk.word_tokenize(line)

In [ ]:
sliced_data = readCorpus()

In order to print generators, we need to iterate through them like a list.

In [ ]:
list(sliced_data)

## Training the Word2Vec Model

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
print "The number of cores available for processing is:" , cores
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

## Building and training our Word2Vec model

Listing the necessary hyperparameteres to tunr our word2Vec model

In [ ]:
num_features = 200 # dimensions of each word embedding
min_word_count = 1 # this is not advisable but since we need to extract
# feature vector for each word we need to do this
num_workers = multiprocessing.cpu_count() # number of threads running in parallel
context_size = 7 # context window length
downsampling = 1e-3 # downsampling for very frequent words
seed = 1 # seed for random number generator to make results reproducible

Now defining our Word2Vec model with the above declared hyperparameters

In [ ]:
word2vec_ = models.Word2Vec(
    sg = 1, seed = seed,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context_size,
    sample = downsampling
)

It's important that we train our vocabulary first before training the model

In [ ]:
word2vec_.build_vocab(sliced_data)

Now training the Word2Vec model with the vocabulary generated above

In [ ]:
word2vec_.train(sliced_data, total_examples = word2vec_.corpus_count, epochs = word2vec_.iter)